# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109A Introduction to Data Science: 

## Homework 3  AC 209 : Regularization


**Harvard University**<br/>
**Fall 2019**<br/>
**Instructors**: Pavlos Protopapas, Kevin Rader and Chris Tanner

<hr style="height:2pt">



In [1]:
# RUN THIS CELL FOR FORMAT
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

In [3]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

%matplotlib inline

<div class='exercise'> <b> Question 1 [12 pts] </b> </div>

Ridge and LASSO regularizations are powerful tools that not only increase generalization, but also expand the range of problems that we can solve. We will study this statement in this question. 

**1.1** Let $X\in \mathbb{R}^{n\times p}$ be a matrix of observations, where each row corresponds an observation and each column corresponds to a predictor. Now consider the case $p > n$: explain why there is no unique solution to the OLS estimator. 

**1.2**  Now consider the Ridge formulation. Show that finding the ridge estimator is equivalent to solving an OLS problem after adding p dummy observations with their X value equal to $\sqrt{\lambda}$ at the j-th component and zero everywhere else, and their Y value set to zero. In a nutshell, show that the ridge estimator can be found by getting the least squares estimator for the augmented problem:

$$X^* = \begin{bmatrix} X \\ \sqrt{\lambda}I \end{bmatrix}$$

$$Y^* = \begin{bmatrix} Y \\ \textbf{0} \end{bmatrix}$$

**1.3** Can we now solve the $p > n$ situation? Explain why.

**1.4** Take a look at the LASSO estimator expression that we derived when $X^TX=I$. What needs to happen for LASSO to nullify  $\beta_i$?

**1.5**  Can LASSO be used when $p>n$? What important consideration, related to the number of predictors that LASSO chooses, do we have to keep in mind in that case?

**1.6** Ridge and LASSO still have room for improvement. List two limitations of Ridge, and two limitations of LASSO.

**1.7** Review the class slides and answer the following questions: When is Ridge preferred? When is LASSO preferred? When is Elastic Net preferred?

### Answers

**1.1 Let $X\in \mathbb{R}^{n\times p}$ be a matrix of observations, where each row corresponds an observation and each column corresponds to a predictor. Now consider the case $p > n$: explain why there is no unique solution to the OLS estimator. **

There is no unique solution to the OLS estimator in the case where p>n because there are an infinite number of possible solutions that minimize the cost/loss function. In order to be effective, OLS requires at least the same number of predictors as observations.

**1.2  Now consider the Ridge formulation. Show that finding the ridge estimator is equivalent to solving an OLS problem after adding p dummy observations with their X value equal to $\sqrt{\lambda}$ at the j-th component and zero everywhere else, and their Y value set to zero. In a nutshell, show that the ridge estimator can be found by getting the least squares estimator for the augmented problem: **

$$X^* = \begin{bmatrix} X \\ \sqrt{\lambda}I \end{bmatrix}$$

$$Y^* = \begin{bmatrix} Y \\ \textbf{0} \end{bmatrix}$$



The least square estimator for the augmented problem above is:
$\beta$ = ($X^*$$^T$$X^*$)$^-1$$X^*$$^T$$Y^*$


Equivalently:
$\beta$ = ([$X$ $\sqrt{\lambda}I$] $\begin{bmatrix} X \\ \sqrt{\lambda}I \end{bmatrix}$)$^-1$[$X$ $\sqrt{\lambda}I$]$\begin{bmatrix} Y \\ \textbf{0} \end{bmatrix}$

Which equals:
$\beta$ = ($X$$^T$$X$+$\lambda$I)$^-1$$X$$^T$$Y$

This last expression is the ridge estimator.

**1.3 Can we now solve the $p > n$ situation? Explain why. **

Yes, we can. The main problem when p>n is that there are an infinite number of solutions that could go through a small number of data points compared to the number of predictors. However, if we penalize high coefficients, then we further constrain the model and limit the number of possible solutions. We can solve the problem using the ridge estimator calculated above.

**1.4 Take a look at the LASSO estimator expression that we derived when $X^TX=I$. What needs to happen for LASSO to nullify  $\beta_i$? **


For LASSO to nullify $\beta_i$, the corresponding $|x_i^Ty|$ has to be smaller than $\lambda^T$ ie. $\lambda/2$


**1.5  Can LASSO be used when $p>n$? What important consideration, related to the number of predictors that LASSO chooses, do we have to keep in mind in that case? **

LASSO can be used in the p>n case. However, if used for p>n, LASSO only uses n predictors and nullifies the remaining (p - n) predictors. When predictors are correlated, LASSO picks one and nullifies the rest, which is why different runs could generate different sets of predictors.   

**5.6 Ridge and LASSO still have room for improvement. List two limitations of Ridge, and two limitations of LASSO. **

Limitations of ridge:
- while Ridge reduces variance in the model, it also adds bias
- doesn't actually discard irrelevant predictors from the model: only minimizes (ie. no feature selection)

Limitations of LASSO:
- when p>n, LASSO uses at most n predictors and discards the rest
- when predictors are correlated, LASSO picks one predictor in an arbitrary way, which means different runs may result in different models

Plus, both are sensible to outliers.

**5.7 Review the class slides and answer the following questions: When is Ridge preferred? When is LASSO preferred? When is Elastic Net preferred? **

Ridge is preferred when we have a high number of informative predictors since, contrary to LASSO, it does not do feature selection. It is also preferred when we would rather shrink correlated predictors over picking one arbitrarily (as is done by LASSO).
LASSO is preferred when we have a powerful predictor which would otherwise be shrunk disproportionately by Ridge and when we have few informative predictors (due to its feature selection capabilities). 
Elastic Net is a good compromise between Ridge and Lasso. It increases stability, reduces model complexity and performs feature selection. For example, it tries to do predictor selection and minimization simultaneously. 




<div class='exercise'><b> Question 2 [12pts]</b></div>

We want to analyze the behavior of our estimators in cases where p > n. We will generate dummy regression problems for this analysis, so that we have full control on the properties of the problem. Sklearn provides an easy to use function to generate regression problems: `sklearn.datasets.make_regression`.

**2.1** Use the provided notebook cell to to build a dataset with 500 samples, 2500 features, 100 informative features and a noise sd of 10.0. The function will return the true coefficients in `true_coef`. Intercepts are not generated, so do not fit them in your regressions. Fit LinearRegression, LassoCV, RidgeCV and ElasticNetCV estimators on the traininig set with 5-fold crossvalidation.

Test 100 lambda values from 0.01 to 1000, in logscale. For Elastic Net, also test the following L1 ratios: [.1, .5, .7, .9, .95, .99] (it is good practice to try more ratio values near the L1 term, as the ridge penalty tends to have higher absolute magnitude).

**Do not change `random_state=209`, to facilitate grading.**

**2.2** As we used `n_informative = 100`, the true betas will contain 100 non-zero values. Let's see if our estimators picked up on that trend. Print the number of betas greater than $10^{-6}$ (non-zero values) for each estimator, and comment on the results.

**2.3**  Let's see how our estimators perform on the test set. Calculate $R^2$ for each estimator on the test set. Comment on the results.

**2.4** Now, let's observe what happens when we  increase the number of informative features. Generate another regression problem with the same parameters as before, but this time with an n_informative of 600. Finally, fit OLS, Ridge, LASSO and EN, and print the number of non-zero coefficients and R2 Scores.


**2.5**  Compare the results with the previous case and comment. What can we say about LASSO and Elastic Net in particular?

In [4]:
# Constants
n= 500
p= 2500
informative= 100
rs = 209
sd = 5

# Generate regresion
X,y,true_coef = make_regression(n_samples = n, n_features = p, n_informative = informative,
                                coef = True, noise = sd)

# Get train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=rs)


### Solutions

**2.1 Use the provided notebook cell to to build a dataset with 500 samples, 2500 features, 100 informative features and a noise sd of 10.0. The function will return the true coefficients in `true_coef`. Intercepts are not generated, so do not fit them in your regressions. Fit LinearRegression, LassoCV, RidgeCV and ElasticNetCV estimators on the traininig set with 5-fold crossvalidation. **

In [5]:
lambdas = np.logspace(-2, 3, 100, base = 10)
L1_ratios = [0.1, 0.5, 0.7, 0.9, 0.95, 0.99] 

print("True Coefficients: {}".format(true_coef))

#fit OLS
lm = LinearRegression(fit_intercept = False)
lm_model = lm.fit(X_train, y_train)
lm_coefficients = lm.coef_
print("OLS Coefficients: {}".format(lm_coefficients))

#fit LassoCV
la = LassoCV(alphas= lambdas, cv = 5, fit_intercept = False)
la_model = la.fit(X_train, y_train)
la_coefficients = la.coef_
print("LASSO Coefficients: {}".format(la_coefficients))

#fit RidgeCV
ri = RidgeCV(alphas = lambdas, cv = 5, fit_intercept = False)
ri_model = ri.fit(X_train, y_train)
ri_coefficients = ri.coef_
print("Ridge Coefficients: {}".format(ri_coefficients))

#fit ElasticNetCV
en = ElasticNetCV(alphas = lambdas, cv = 5, fit_intercept = False, l1_ratio = L1_ratios, max_iter = 10000)
en_model = en.fit(X_train, y_train)
en_coefficients = en.coef_
print("Elastic Net Coefficients: {}".format(en_coefficients))

True Coefficients: [0. 0. 0. ... 0. 0. 0.]
OLS Coefficients: [ 2.49532172 -3.88109851 -1.05746322 ...  3.43275122 -1.93670282
  5.96516204]
LASSO Coefficients: [ 0.         -3.2768493  -0.         ...  0.         -0.
  3.72692421]
Ridge Coefficients: [ 2.4953063  -3.88107855 -1.05744828 ...  3.43273865 -1.93670633
  5.96513758]
Elastic Net Coefficients: [ 0. -0.  0. ...  0. -0.  0.]


**2.2 As we used `n_informative = 100`, the true betas will contain 100 non-zero values. Let's see if our estimators picked up on that trend. Print the number of betas with absolute value greater than $10^{-6}$ (which will corrspond to non-zero values) for each estimator, and comment on the results. **

In [6]:
th = 1e-6 #threshold

#counts keeping track of how many beta estimates exceed the threshold for each model
lm_coef_count = 0
la_coef_count = 0
ri_coef_count = 0
en_coef_count = 0
true_coef_count = 0

#loops through arrays of coefficients and counts number of coefficients who's absolute values exceed the threshold
for coef in lm_coefficients:
    if abs(coef) > th:
        lm_coef_count += 1
        
for coef in la_coefficients:
    if abs(coef) > th:
        la_coef_count += 1

for coef in ri_coefficients:
    if abs(coef) > th:
        ri_coef_count += 1

for coef in en_coefficients:
    if abs(coef) > th:
        en_coef_count += 1
        
for coef in true_coef:
    if abs(coef) > th:
        true_coef_count += 1
        
#print statements
print("Number of Betas with absolute value greater than 1e-6 for OLS: {}".format(lm_coef_count))
print("Number of Betas with absolute value greater than 1e-6 for LASSO: {}".format(la_coef_count))
print("Number of Betas with absolute value greater than 1e-6 for Ridge: {}".format(ri_coef_count))
print("Number of Betas with absolute value greater than 1e-6 for Elastic Net: {}".format(en_coef_count))
print("Number of Betas with absolute value greater than 1e-6 for Actual Data: {}".format(true_coef_count))


Number of Betas with absolute value greater than 1e-6 for OLS: 2500
Number of Betas with absolute value greater than 1e-6 for LASSO: 315
Number of Betas with absolute value greater than 1e-6 for Ridge: 2500
Number of Betas with absolute value greater than 1e-6 for Elastic Net: 324
Number of Betas with absolute value greater than 1e-6 for Actual Data: 100


From the results above, we see that LASSO and Elastic Net are the only two models that do feature selection and actually nullify irrelevant predictors. LASSO is the one that gets closest to the 100 from the actual data set (ie. 282 predictors) and Elastic Net is a compromise between LASSO and Ridge and thus less selective than the LASSO model (ie. 648 predictors). Ridge and OLS fail to nullify predictors altogether (ie. keep all 2500 predictors).



**2.3**  Let's see how our estimators perform on the test set. Calculate $R^2$ for each estimator on the test set. Comment on the results.

In [7]:
lm_predictions = lm.predict(X_test)
lm_r2 = r2_score(y_test, lm_predictions)
print("R^2 score for OLS is: {}".format(lm_r2))

la_predictions = la.predict(X_test)
la_r2 = r2_score(y_test, la_predictions)
print("R^2 score for LASSO is: {}".format(la_r2))

ri_predictions = ri.predict(X_test)
ri_r2 = r2_score(y_test, ri_predictions)
print("R^2 score for Ridge is: {}".format(ri_r2))

en_predictions = en.predict(X_test)
en_r2 = r2_score(y_test, en_predictions)
print("R^2 score for Elastic Net is: {}".format(en_r2))


R^2 score for OLS is: 0.07663861080466006
R^2 score for LASSO is: 0.6934720987576901
R^2 score for Ridge is: 0.07663863662700066
R^2 score for Elastic Net is: 0.46975188657566647


The model with the best $R^2$ score is LASSO and the worst ones are OLS and Ridge. This is expected since LASSO nullifies features that are not as relevant or not relevant while Ridge only shrinks them. As such, we'd expect LASSO to fit the dataset better. Elastic Net is a compromise between Ridge and LASSO and thus has an $R^2$ score in between the two.

**2.4 Now, let's observe what happens when we  increase the number of informative features. Generate another regression problem with the same parameters as before, but this time with an n_informative of 600. Finally, fit OLS, Ridge, LASSO and EN, and print the number of non-zero coefficients and R2 Scores. **

In [8]:
informative_new= 600

# Generate regresion
X,y,true_coef = make_regression(n_samples = n, n_features = p, n_informative = informative_new,
                                coef = True, noise = sd)

# Get train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=rs)

print("True Coefficients: {}".format(true_coef))


##Fit models
#fit OLS
lm = LinearRegression(fit_intercept = False)
lm_model = lm.fit(X_train, y_train)
lm_coefficients = lm.coef_
print("OLS Coefficients: {}".format(lm_coefficients))

#fit LassoCV
la = LassoCV(alphas= lambdas, cv = 5, fit_intercept = False)
la_model = la.fit(X_train, y_train)
la_coefficients = la.coef_
print("LASSO Coefficients: {}".format(la_coefficients))

#fit RidgeCV
ri = RidgeCV(alphas = lambdas, cv = 5, fit_intercept = False)
ri_model = ri.fit(X_train, y_train)
ri_coefficients = ri.coef_
print("Ridge Coefficients: {}".format(ri_coefficients))

#fit ElasticNetCV
en = ElasticNetCV(alphas = lambdas, cv = 5, fit_intercept = False, l1_ratio = L1_ratios, max_iter = 10000)
en_model = en.fit(X_train, y_train)
en_coefficients = en.coef_
print("Elastic Net Coefficients: {}".format(en_coefficients))


##Number of Betas above 1e-6 threshold
th = 1e-6 #threshold

#counts keeping track of how many beta estimates exceed the threshold for each model
lm_coef_count = 0
la_coef_count = 0
ri_coef_count = 0
en_coef_count = 0
true_coef_count = 0

#loops through arrays of coefficients and counts number of coefficients who's absolute values exceed the threshold
for coef in lm_coefficients:
    if abs(coef) > th:
        lm_coef_count += 1
        
for coef in la_coefficients:
    if abs(coef) > th:
        la_coef_count += 1

for coef in ri_coefficients:
    if abs(coef) > th:
        ri_coef_count += 1

for coef in en_coefficients:
    if abs(coef) > th:
        en_coef_count += 1
        
for coef in true_coef:
    if abs(coef) > th:
        true_coef_count += 1
        
#print statements
print("Number of Betas with absolute value greater than 1e-6 for Linear Regression: {}".format(lm_coef_count))
print("Number of Betas with absolute value greater than 1e-6 for LASSO: {}".format(la_coef_count))
print("Number of Betas with absolute value greater than 1e-6 for Ridge: {}".format(ri_coef_count))
print("Number of Betas with absolute value greater than 1e-6 for Elastic Net: {}".format(en_coef_count))
print("Number of Betas with absolute value greater than 1e-6 for Actual Data: {}".format(true_coef_count))



##R^2 scores
lm_predictions = lm.predict(X_test)
lm_r2 = r2_score(y_test, lm_predictions)
print("R^2 score for OLS is: {}".format(lm_r2))

la_predictions = la.predict(X_test)
la_r2 = r2_score(y_test, la_predictions)
print("R^2 score for LASSO is: {}".format(la_r2))

ri_predictions = ri.predict(X_test)
ri_r2 = r2_score(y_test, ri_predictions)
print("R^2 score for Ridge is: {}".format(ri_r2))

en_predictions = en.predict(X_test)
en_r2 = r2_score(y_test, en_predictions)
print("R^2 score for Elastic Net is: {}".format(en_r2))




True Coefficients: [ 0.         52.19919716  0.         ...  0.         75.92265075
  0.        ]
OLS Coefficients: [ 7.55308533  9.87807871 19.94142677 ... 15.6907177  20.38565803
  4.35790212]
LASSO Coefficients: [0. 0. 0. ... 0. 0. 0.]
Ridge Coefficients: [ 4.93303011  6.21455076 14.81498734 ... 12.02175632 13.77286597
  3.07392367]
Elastic Net Coefficients: [ 4.52002447  5.95294536 15.31407695 ... 12.0871949  14.32904183
  2.61505037]
Number of Betas with absolute value greater than 1e-6 for Linear Regression: 2500
Number of Betas with absolute value greater than 1e-6 for LASSO: 29
Number of Betas with absolute value greater than 1e-6 for Ridge: 2500
Number of Betas with absolute value greater than 1e-6 for Elastic Net: 2302
Number of Betas with absolute value greater than 1e-6 for Actual Data: 600
R^2 score for OLS is: 0.20947181541155235
R^2 score for LASSO is: 0.03963918103235031
R^2 score for Ridge is: 0.1741048597324616
R^2 score for Elastic Net is: 0.17479361406155858


**2.5  Compare the results with the previous case and comment. What can we say about LASSO and Elastic Net in particular? **

Comparing this case (informative predictors = 600) with the previous one (informative predictors = 100), we can see that LASSO now has a negative $R^2$ value and Elastic Net now has the highest $R^2$ value. In other words, LASSO is the worst model when compared to OLS, Ridge and EN in the case where there are more informative predictors. This is expected because LASSO performs feature selection and thus nullifies lots of informative predictors (only 40 out of 600 remain) while Ridge and OLS only shrink them (keep all 2500). Elastic Net gives us the best of both worlds between Ridge and LASSO and thus has the best $R^2$ score with a large number of informative predictors (only keeps 2479 out of 2500). While it isn't a perfect model, it is the best out of the 4 we tested.


<div class='exercise'><b> Question 3 [1pt] (for fun) </b></div>

We would like to visualize how Ridge, LASSO and Elastic Net behave. We will build a toy regression example to observe the behavior of the coefficients and loss function as lambda increases.

**3.1** Use `sklearn.datasets.make_regression` to build a well-conditioned regression problem with 1000 samples, 5 features, noise standard deviation of 10 and random state 209.

**3.2** Find the Ridge, LASSO and EN estimator for this problem, varying the regularization parameter in the interval $[0.1,100]$ for LASSO and EN, and $[0.1,10000]$ for Ridge. Plot the evolution of the 5 coefficients for each estimator in a 2D plot, where the X axis is the regularization parameter and the Y axis is the coefficient value. For Elastic Net, make 4 plots, each one with one of the following L1 ratios: $[0.1, 0.5, 0.8, 0.95]$ You should have 6 plots: one for Lasso, one for Ridge, and 4 for EN. Each plot should have 5 curves, one per coefficient. 

**3.3** Comment on this evolution. Does this make sense with what we've seen so far?

**3.4** We're now interested in visualizing the behavior of the Loss functions. First, generate a regression problem with 1000 samples and 2 features. Then, use the provided "loss_3d_interactive" function to observe how the loss surface changes as the regularization parameter changes. Test the function with Ridge_loss, LASSO_loss and EN_loss. Comment on what you observe.**

**Note: for this to work, you have to install plotly. Go to https://plot.ly/python/getting-started/ and follow the steps. You don't need to make an account as we'll use the offline mode.**

### Solutions 

**3.1 Use `sklearn.datasets.make_regression` to build a well-conditioned regression problem with 1000 samples, 5 features, noise standard deviation of 10 and random state 209. **



In [6]:
# your code here 


**3.2 Find the Ridge, LASSO and EN estimator for this problem, varying the regularization parameter in the interval $[0.1,100]$ for LASSO and EN, and $[0.1,10000]$ for Ridge. Plot the evolution of the 5 coefficients for each estimator in a 2D plot, where the X axis is the regularization parameter and the Y axis is the coefficient value. For Elastic Net, make 4 plots, each one with one of the following L1 ratios: $[0.1, 0.5, 0.8, 0.95]$ You should have 6 plots: one for Lasso, one for Ridge, and 4 for EN. Each plot should have 5 curves, one per coefficient. **

In [7]:
# your code here 


In [8]:
#your code here


**3.3 Comment on this evolution. Does this make sense with what we've seen so far?**

*your answer here* 



**3.4 We're now interested in visualizing the behavior of the Loss functions. First, generate a regression problem with 1000 samples and 2 features. Then, use the provided "loss_3d_interactive" function to observe how the loss surface changes as the regularization parameter changes. Test the function with Ridge_loss, LASSO_loss and EN_loss. Comment on what you observe.**

**Note: for this to work, you have to install plotly. Go to https://plot.ly/python/getting-started/ and follow the steps. You don't need to make an account as we'll use the offline mode.**

In [61]:
X,y,true_coef = make_regression(n_samples = 1000, n_features = 2, noise = 10, random_state=209, coef=True)

In [62]:
from ipywidgets import interactive, HBox, VBox
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

def OLS_loss(X, y, beta, lbda=0):
    y_hat = np.dot(X,beta)
    return np.sum((y_hat-y)**2,axis=0)

def Ridge_loss(X, y, beta, lbda):
    y_hat = np.dot(X,beta)
    return np.sum((y_hat-y)**2,axis=0) + lbda*np.sum(beta**2, axis=0)

def LASSO_loss(X, y, beta, lbda):
    y_hat = np.dot(X,beta)
    return (1 / (2 * len(X)))*np.sum((y_hat-y)**2,axis=0) + lbda*np.sum(np.abs(beta), axis=0)

def EN_loss(X, y, beta, lbda):
    ratio=0.1
    y_hat = np.dot(X,beta)
    return (1 / (2 * len(X)))*np.sum((y_hat-y)**2,axis=0) + lbda*(ratio*np.sum(beta**2, axis=0) + (1-ratio)*np.sum(np.abs(beta), axis=0))

def loss_3d_interactive(X, y, loss='Ridge'):
    '''Uses plotly to draw an interactive 3D representation of the loss function, 
    with a slider to control the regularization factor.
    
    Inputs:
    X: predictor matrix for the regression problem. Has to be of dim n x 2
    y: response vector 
    
    loss: string with the loss to plot. Options are 'Ridge', 'LASSO', 'EN'.
    '''
    
    if loss == 'Ridge':
        loss_function = Ridge_loss
        lbda_slider_min = 0
        lbda_slider_max = 10000
        lbda_step = 10
        clf = Ridge()
    elif loss == 'LASSO':
        loss_function = LASSO_loss
        lbda_slider_min = 1
        lbda_slider_max = 150
        lbda_step = 1
        clf = Lasso()
    elif loss == 'EN':
        loss_function = EN_loss
        lbda_slider_min = 1
        lbda_slider_max = 150
        lbda_step = 1
        clf = ElasticNet()
    else:
        raise ValueError("Loss string not recognized. Available options are: 'Ridge', 'LASSO', 'EN'.")
        
    
    # linspace for loss surface
    L=20
    lsp_b = np.linspace(-80,80,L)
    lsp_b_x, lsp_b_y = np.meshgrid(lsp_b,lsp_b)
    lsp_b_mat = np.column_stack((lsp_b_x.flatten(),lsp_b_y.flatten()))
    
    # Get all optimal betas for current lambda range
    precomp_coefs=[]
    for l in range(lbda_slider_min,lbda_slider_max+1,lbda_step):
        clf.set_params(alpha=l)
        clf.fit(X, y)
        precomp_coefs.append(clf.coef_)
                
    f = go.FigureWidget(
        data=[
            go.Surface(
                    x=lsp_b_x,
                    y=lsp_b_y,
                    z=loss_function(X,y.reshape(-1,1), lsp_b_mat.T, 0).reshape((L,L)),
                    colorscale='Viridis',
                    opacity=0.7,
                    contours=dict(z=dict(show=True,
                                         width=3,
                                         highlight=True,
                                         highlightcolor='orange',
                                         project=dict(z=True),
                                         usecolormap=True))
            ),
            
            go.Scatter3d(
                x=[p[0] for p in precomp_coefs],
                y=[p[1] for p in precomp_coefs],
                z=np.zeros(len(precomp_coefs)),
                marker=dict(
                    size=1,
                    color='darkorange',
                    line=dict(
                        color='darkorange',
                        width=1
                        ),
                    opacity=1
                    )
                ),
            go.Scatter3d(
                x=[0],
                y=[0],
                z=[0],
                
                marker=dict(
                    size=10,
                    color='orange',
                    opacity=1
                    ),
            )
        ],

        layout=go.Layout(scene=go.layout.Scene(
                    xaxis = dict(
                        title='Beta 1'),
                    yaxis = dict(
                        title='Beta 2'),
                    zaxis = dict(
                        title='Loss'),
            camera=go.layout.scene.Camera(
                up=dict(x=0, y=0, z=1),
                center=dict(x=0, y=0, z=0),
                eye=dict(x=1.25, y=1.25, z=1.25))
        ),
            width=1000,
            height=700,)
    )

    def update_z(lbda):
        f.data[0].z = loss_function(X, y.reshape(-1,1), lsp_b_mat.T, lbda).reshape((L,L))
        beta_opt = precomp_coefs[(lbda-lbda_slider_min)//(lbda_step)]
        f.data[-1].x = [beta_opt[0]]
        f.data[-1].y = [beta_opt[1]]
        f.data[-1].z = [0]

    lambda_slider = interactive(update_z, lbda=(lbda_slider_min, lbda_slider_max, lbda_step))
    vb = VBox((f, lambda_slider))
    vb.layout.align_items = 'center'
    display(vb)
    

In [10]:
#your code here


In [11]:
#your code here


In [12]:
#your code here
